드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


작업 디렉토리 설정 

In [ ]:
%cd /content/drive/MyDrive/SW_capstone/PCN-main/FISH_NET

/content/drive/MyDrive/SW_capstone/PCN-main/FISH_NET


필요 모듈 다운로드 

In [ ]:
pip install tensorboardX

In [ ]:
pip install matplotlib

In [ ]:
pip install easydict

이미지 데이터 zip file 압축 해제 

In [ ]:
%cd /content/drive/MyDrive/SW_capstone/PCN-main/dataset_3/data_3/test/

!unzip -qq "/content/drive/MyDrive/SW_capstone/PCN-main/dataset_3/data_3/test/test.zip"

In [ ]:
%cd /content/drive/MyDrive/SW_capstone/PCN-main/dataset_3/gt_3/train/

!unzip -qq "/content/drive/MyDrive/SW_capstone/PCN-main/dataset_3/gt_3/train/train.zip"

flist 파일 생성 

In [ ]:
!python /content/drive/MyDrive/SW_capstone/PCN-main/dataset_3/flist.py

Train

In [ ]:
!python /content/drive/MyDrive/SW_capstone/PCN-main/FISH_NET/train.py

Test

In [ ]:
!python /content/drive/MyDrive/SW_capstone/PCN-main/FISH_NET/test.py

특정 이미지 실험 및 저장

In [ ]:
import torch
import torch.multiprocessing as mp
from torch.utils.data import DataLoader
import torchvision.transforms as tf

from PIL import Image
import math
import os
import importlib
import datetime
import json
import easydict
import numpy as np 

### My libs
from core.utils import set_device, set_seed
from core.utils import postprocess
from core.dataset import Dataset

args = easydict.EasyDict(
  {
    "config": '/content/drive/MyDrive/SW_capstone/PCN-main/FISH_NET/configs/places2.json', 
    "model_name": 'pennet4', 
    "level" : None,
    "mask": 'square', 
    "size": 256, 
    "exam": 'store_true'
  }
)

strong_path = '/content/drive/MyDrive/SW_capstone/PCN-main/dataset_2/'
weak_path = '/content/drive/MyDrive/SW_capstone/PCN-main/dataset/'

#real_image_1 = '/content/drive/MyDrive/SW_capstone/PCN-main/real_images/00009_RV.png'
real_image_2 = '/content/drive/MyDrive/SW_capstone/PCN-main/real_images/real_image_2_cut.jpg'
#real_image_3 = '/content/drive/MyDrive/SW_capstone/PCN-main/real_images/real_image_2.jpg'

#gt =  Image.open(weak_path + 'gt/test/17.jpg')
#image1 = Image.open(weak_path + 'data/test/17.jpg')
#image2 = Image.open('/content/drive/MyDrive/SW_capstone/PCN-main/real_images/real_image_2.jpg')

#real_image_1 = Image.open(real_image_1)
rel_image_2 = Image.open(real_image_2)
#real_image_3 = Image.open(real_image_3)

#real_image_1 = real_image_1.resize((256,256))
real_image_2 = real_image_2.resize((256,256))
#real_image_3 = real_image_3.resize((256,256))

#real_image_1 = real_image_1.convert('RGB')
real_image_2 = real_image_2.convert('RGB')
#real_image_3 = real_image_3.convert('RGB')


#real_image_1.save('/content/drive/MyDrive/SW_capstone/PCN-main/FISH_NET/release_model_0508/pennet4_dataset_3_square256/result_real_image/1.jpg')
real_image_2.save('/content/drive/MyDrive/SW_capstone/PCN-main/FISH_NET/release_model_0426/pennet4_dataset_2_square256/result_real_image/real_image_2_cut_resize.jpg')
#real_image_3.save('/content/drive/MyDrive/SW_capstone/PCN-main/FISH_NET/release_model_0508/pennet4_dataset_3_square256/result_real_image/3.jpg')

totensor = tf.ToTensor()

#real_image_1_tensor = totensor(real_image_1)
#real_image_1_tensor = real_image_1_tensor * 2 - 1 
#real_image_1_tensor = real_image_1_tensor.unsqueeze(0)

real_image_2_tensor = totensor(real_image_2)
real_image_2_tensor = real_image_2_tensor * 2 - 1 
real_image_2_tensor = real_image_2_tensor.unsqueeze(0)

#real_image_3_tensor = totensor(real_image_3)
#real_image_3_tensor = real_image_3_tensor * 2 - 1 
#real_image_3_tensor = real_image_3_tensor.unsqueeze(0)


config = json.load(open(args.config))

net = importlib.import_module('model.'+args.model_name)
model = set_device(net.InpaintGenerator())
path = os.path.join(config['save_dir'], 'pennet4_dataset_2_square256', 'gen_00030.pt')
data = torch.load(path, map_location = lambda storage, loc: set_device(storage)) 
model.load_state_dict(data['netG'])
model.eval()

#real_image_1_tensor = set_device(real_image_1_tensor)
#_, output1 = model(real_image_1_tensor)  
#pred_imgs1 = postprocess(output1)

#Image.fromarray(pred_imgs1[0]).save('/content/drive/MyDrive/SW_capstone/PCN-main/FISH_NET/release_model_0508/pennet4_dataset_3_square256/result_real_image/1_undistort.jpg')

real_image_2_tensor = set_device(real_image_2_tensor)
_, output2 = model(real_image_2_tensor)
pred_imgs2 = postprocess(output2)

Image.fromarray(pred_imgs2[0]).save('/content/drive/MyDrive/SW_capstone/PCN-main/FISH_NET/release_model_0426/pennet4_dataset_2_square256/result_real_image/real_image_2_cut_resize_undistort.jpg')

#real_image_3_tensor = set_device(real_image_3_tensor)
#_, output3 = model(real_image_3_tensor)
#pred_imgs3 = postprocess(output3)

#Image.fromarray(pred_imgs3[0]).save('/content/drive/MyDrive/SW_capstone/PCN-main/FISH_NET/release_model_0508/pennet4_dataset_3_square256/result_real_image/3_undistort.jpg')



Test(한 장씩 test)

In [ ]:
import torch
import torch.multiprocessing as mp
from torch.utils.data import DataLoader
import torchvision.transforms as tf

from PIL import Image
import math
import os
import importlib
import datetime
import json
import easydict
import numpy as np 

### My libs
from core.utils import set_device, set_seed
from core.utils import postprocess
from core.dataset import Dataset

args = easydict.EasyDict(
  {
    "config": '/content/drive/MyDrive/SW_capstone/PCN-main/FISH_NET/configs/places2.json', 
    "model_name": 'pennet4', 
    "level" : None,
    "mask": 'square', 
    "size": 256, 
    "exam": 'store_true'
  }
)

config = json.load(open(args.config))

net = importlib.import_module('model.'+args.model_name)
model = set_device(net.FlowColumn())
path = os.path.join(config['save_dir'], 'pennet4_dataset_3_square256', 'gen_00050.pt')
data = torch.load(path, map_location = lambda storage, loc: set_device(storage))
model.load_state_dict(data['netG'], strict = False)
model.eval()

strong_path = '/content/drive/MyDrive/SW_capstone/PCN-main/dataset_2/'
weak_path = '/content/drive/MyDrive/SW_capstone/PCN-main/dataset/'
medium_path = '/content/drive/MyDrive/SW_capstone/PCN-main/dataset_3/'

for i in range(1, 10001):
  #gt =  Image.open(weak_path + 'gt/test/' + str(i) + '.jpg')
  image1 = Image.open(medium_path + 'data_3/test/' + str(i) + '.jpg')

  image1 = image1.resize((256,256))
  image1 = image1.convert('RGB')

  #gt = gt.resize((256,256))
  #gt = gt.convert('RGB')

  #gt.save('/content/drive/MyDrive/SW_capstone/PCN-main/FISH_NET/release_model_0508/pennet4_dataset_3_square256/50_result_weak/'+str(i) +'_gt_256.jpg')
  #image1.save('/content/drive/MyDrive/SW_capstone/PCN-main/FISH_NET/release_model_0508/pennet4_dataset_3_square256/50_result_weak/' + str(i) + '_data_256.jpg')
  totensor = tf.ToTensor()

  image1_tensor = totensor(image1)
  image1_tensor = image1_tensor * 2 - 1 
  image1_tensor = image1_tensor.unsqueeze(0)


  image1_tensor = set_device(image1_tensor)
  _, output1 = model(image1_tensor)
  #_ = (_+1)/2*255
  _ = _.int().cpu().numpy()
  pred_imgs1 = np.array(_[0][0])
  pred_imgs1 = pred_imgs1 * 255
  pred_imgs1 = np.reshape(pred_imgs1, (256,256,1))
  pred_imgs2 = np.array(_[0][1])
  pred_imgs2 = pred_imgs2 * 255
  pred_imgs2 = np.reshape(pred_imgs2, (256,256,1))
  pred_imgs3 = (pred_imgs1 + pred_imgs2) / 2 

  new = np.concatenate((pred_imgs1, pred_imgs1), axis = 2)
  new = np.concatenate((new, pred_imgs1), axis = 2).astype(np.uint8)

  new2 = np.concatenate((pred_imgs2, pred_imgs2), axis = 2)
  new2 = np.concatenate((new2, pred_imgs2), axis = 2).astype(np.uint8)

  new3 = new + new2



  Image.fromarray(new3).save('/content/drive/MyDrive/SW_capstone/PCN-main/FISH_NET/release_model_0508/pennet4_dataset_3_square256/50_result/' + str(i) + '_flowmap_3_256.jpg')
  print(i, "has been rectificated")



evaluation metric 이용 단락

In [ ]:
import torch
import torch.multiprocessing as mp
from torch.utils.data import DataLoader
import torchvision.transforms as tf

from PIL import Image
import math
import os
import importlib
import datetime
import json
import easydict
import numpy as np 

from core.metric import psnr, ssim, mae


image_path = '/content/drive/MyDrive/SW_capstone/PCN-main/FISH_NET/release_model_0508/pennet4_dataset_3_square256/50_result/'
#image_path_2 = '/content/drive/MyDrive/SW_capstone/PCN-main/FISH_NET/release_model_0508/pennet4_dataset_3_square256/result/'

image_lst = os.listdir(image_path)
#image_lst_2 = os.listdir(image_path_2)

psnr_score = 0
ssim_score = 0
mae_score = 0

for i in range(1,10001):
  PSNR = 0 
  SSIM = 0
  MAE = 0

  gt = Image.open(image_path + str(i) + '_gt_256.jpg')
  res = Image.open(image_path + str(i) + '_undistort_256.jpg')
  #res_2 = Image.open(image_path_2 + str(i) + '_undistort_256.jpg')

  gt = np.array(gt)
  res = np.array(res)
  #res_2 = np.array(res_2)

  PSNR = psnr(res, gt)
  SSIM = ssim(res, gt)
  MAE = mae(res, gt)

  psnr_score = psnr_score + PSNR
  ssim_score = ssim_score + SSIM
  mae_score = mae_score + MAE

  print(i, " has finished")

psnr_score = psnr_score / 100 
ssim_score = ssim_score / 100
mae_score = mae_score / 100

print("PSNR : ", psnr_score)
print("SSIM : ", ssim_score)
print("MAE : ", mae_score)

  

Tensorboard 확인

In [ ]:
%load_ext tensorboard